### predictions

### import modules and configure notebook

In [18]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

from functions.model_data import bestHyperparamaters 
from functions.postproccessing import process_results

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats
%store -r X_test_labeled_df



no stored variable best_feats_ranks


### configurations
* save_plots -> True|False
* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [3]:
save_plots = False
save_predictions = False

In [4]:
from config import *

### counts of instances in all classes before oversampling

In [5]:
train_data_formodel['class'].value_counts()

4     105
17    100
18     61
0      53
10     47
13     45
15     36
16     36
2      36
12     30
11     30
8      30
7      30
5      30
6      27
9      27
1      24
14     21
3      18
Name: class, dtype: int64

### The class column is stored as the variable y and the variables identified as best by the 2 feature_selection notebook are used as features

In [6]:
y = np.array(train_data_formodel[target])
X = np.array(np.array(train_data_formodel[best_feats]))

### the dimensions of the class and features are checked

In [7]:
print(X.shape)
print(y.shape)

(786, 25)
(786,)


### grid search is utilised to identify optimum hyperparamaters

In [8]:
my_optimiser = bestHyperparamaters(X, y)
my_optimiser.get_best_model()

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    3.2s finished
/Users/robbie.morse/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [9]:
best_model = my_optimiser.best_model

### Model is built for predicting source of artefacts 

In [10]:
best_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [11]:
print(best_model)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)


In [12]:
print(train_data_formodel[target].unique())
print(uniques)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Index(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG', 'WB_BX',
       'PF', 'BM', 'WH', 'SQ', 'WN', 'BH', 'PH', 'LB'],
      dtype='object')


In [13]:
identifiers =  X_test_labeled_df['Analysis']

In [14]:
results = process_results(best_model, X_test_labeled_df, best_feats, uniques, identifiers)

In [15]:
results.head()

,class_number,FH,ER,WW,TC,CS,BC,KQ,AR,SL,FG,WB_BX,PF,BM,WH,SQ,WN,BH,PH,LB,Analysis,inlierLabel,class_predictions
0,1,0.0,0.400000,0.000000,0.025806,0.216129,0.0,0.0,0.019355,0.0,0.0,0.139785,0.019355,0.0,0.012903,0.0,0.009677,0.012903,0.066667,0.077419,06_DH1_1,1,ER
1,16,0.0,0.200000,0.000000,0.025806,0.016129,0.0,0.0,0.019355,0.0,0.2,0.139785,0.019355,0.0,0.012903,0.0,0.009677,0.212903,0.066667,0.077419,07_DH1_2,1,BH
2,18,0.0,0.066667,0.133333,0.025806,0.016129,0.0,0.0,0.019355,0.0,0.0,0.006452,0.019355,0.0,0.012903,0.0,0.009677,0.212903,0.200000,0.277419,08_DH1_3,1,LB
3,9,0.0,0.100000,0.300000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,09_DH2_1,-1,other
4,9,0.0,0.000000,0.200000,0.200000,0.000000,0.0,0.0,0.000000,0.0,0.4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.200000,0.000000,10_DH2_2,1,FG


### predictions are stored as a variable into memory

In [16]:
%store results

Stored 'results' (DataFrame)


### predictions are outputted as csv file

In [17]:
if save_predictions:
    results.to_csv('output_datasets/predictions.csv')
